In [1]:
from mesa import Agent, Model
from mesa.time import RandomActivation, BaseScheduler
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, Point, MultiPoint
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy import exp
from numpy.random import rand, seed
import folium
import random 
from haversine import haversine
pd.options.mode.chained_assignment = None  # default='warn'

In [430]:
class ConstructionSite(Agent): 
    def __init__(self, unique_id, model, buildingType, coords, inA10):
        super().__init__(unique_id, model)
        self.buildingType = buildingType # A, B, C...etc. 
        self.coords = coords 
        self.inA10 = inA10 # True or False 
        self.material_request = {}
        self.materials_received = {}
        
        self.calc_materials_required()
            
    def calc_materials_required(self): 
        materials_required = {}
        b = self.model.build_info.copy()
        b = b[(b.buildingType == self.buildingType) & 
              (b.biobased == self.model.biobased_type)] 
        
        if self.model.modularity_type == 'none': 
            mat_list = self.model.material_list.copy()
            mat_list.remove('modules')
            materials_required = dict.fromkeys(mat_list,0)
            for mat in mat_list: 
                for strucType in ['structural', 'nonstructural']: 
                    b1 = b[b.strucType == strucType]
                    materials_required[mat] += b1[mat].iloc[0]
        else: 
            mat_list = self.model.material_list.copy() 
            materials_required = dict.fromkeys(mat_list,0)
            for mat in mat_list: 
                b1 = b[b.strucType == 'structural']
                materials_required[mat] += b1[mat].iloc[0]
                
        self.materials_required = materials_required
        self.materials_received = dict.fromkeys(materials_required.keys(), 0)
        
    def step(self): 
        self.list_materials_toRequest()
        self.request_materials() 
    
    def list_materials_toRequest(self): 
        materials_toRequest = []
        for mat in self.materials_required.keys():  
            mat_required = self.materials_required[mat]
            mat_received = self.materials_received[mat]
            if mat_received < mat_required: 
                materials_toRequest.append(mat)
        self.materials_toRequest_list = materials_toRequest
        
    def request_materials(self):
        material_request = {}
        for mat in self.materials_toRequest_list:
            mat_required = self.materials_required[mat]
            mat_received = self.materials_received[mat] 
            mat_stillNeeded = mat_required - mat_received
            mat_request = round(mat_required * 0.1) 
            material_request[mat] = mat_request if mat_stillNeeded > mat_request else mat_stillNeeded
        self.material_request = material_request

class Hub(Agent):
    def __init__(self, unique_id, model, hubType, coords, inA10):
        super().__init__(unique_id, model)
        self.hubType = hubType # macro or micro 
        self.coords = coords
        self.inA10 = inA10
        self.nearestMacroHub_id = None
        self.nearestMacroHub_dist = None
        self.mat_to_send = {}
        self.nTrips = {}
        self.material_request = {}
        self.materials_received = {}

    def step(self):
        self.find_assigned_sites()
        if self.assigned_sites: # if assigned_sites list is not empty:
            self.calc_mat_to_send() # calculate material to send to each site
            self.request_materials()
            self.calc_n_trips() # calculate number of trips required for truck
            self.move_materials()
        
    def find_assigned_sites(self): 
        if self.model.hub_network == 'decentralized': 
            if self.hubType == 'macro': 
                hubs = [hub for hub in self.model.hubs if hub.hubType == 'micro']
                assigned_sites = [hub for hub in hubs if hub.nearestMacroHub_id == self.unique_id]
            elif self.hubType == 'micro': 
                sites = self.model.construction_sites
                assigned_sites = [site for site in sites if site.nearestMicroHub_id == self.unique_id]
        elif self.model.hub_network == 'centralized': 
            sites = self.model.construction_sites
            assigned_sites = [site for site in sites if site.nearestMacroHub_id == self.unique_id]
        self.assigned_sites = assigned_sites
        
    def calc_mat_to_send(self): 
        sites = self.assigned_sites
        mat_to_send = {}
        for site in sites: 
            if sum(site.material_request.values()) > 0: 
                mat_to_send[site.unique_id] = site.material_request
        self.mat_to_send = mat_to_send
        
    def request_materials(self):
        total_mat_amounts = {}
        for site_id, mat_amounts in self.mat_to_send.items():
            for mat, amount in mat_amounts.items(): 
                try: 
                    total_mat_amounts[mat] += amount 
                except: 
                    total_mat_amounts[mat] = 0
                    total_mat_amounts[mat] += amount 
        self.material_request = total_mat_amounts

    def calc_n_trips(self): 
        trucks = self.model.trucks_toSite
        truck = [truck for truck in trucks if truck.hub_id == self.unique_id][0]
        for dest_id, mat_amounts in self.mat_to_send.items(): 
            nTrips_list = []
            for mat, amounts in mat_amounts.items(): 
                capacity = truck.capacity[mat]
                nTrips = math.ceil(amounts / capacity)
                nTrips_list.append(nTrips)
            self.nTrips[dest_id] = sum(nTrips_list)
    
    def move_materials(self): 
        if self.model.hub_network == 'decentralized': 
            if self.hubType == 'micro': 
                od = self.model.od_matrix_h2c 
            elif self.hubType == 'macro': 
                od = self.model.od_matrix_h2h
        elif self.model.hub_network == 'centralized': 
            od = self.model.od_matrix_h2c
        trucks = self.model.trucks_toSite
        truck = [truck for truck in trucks if truck.hub_id == self.unique_id][0]
        emissions_perKm = truck.emissions_perKm 
        emissions_list = []
        for dest_id, mat_amounts in self.mat_to_send.items(): 
            row = od[(od[:,0] == dest_id) & (od[:,1] == self.unique_id)]
            distance = row[0][2]
            emissions = distance * self.nTrips[dest_id] * emissions_perKm
            emissions_list.append(emissions)
            sites = self.assigned_sites
            dest = [site for site in sites if site.unique_id == dest_id][0]
            for mat, amount in mat_amounts.items(): 
                try: 
                    dest.materials_received[mat] += amount
                except: 
                    dest.materials_received[mat] = 0 
                    dest.materials_received[mat] += amount
        truck.emissions += sum(emissions_list)
        
class Supplier(Agent): 
    def __init__(self, unique_id, model, material, distFromAms, coords): 
        super().__init__(unique_id, model)
        self.material = material
        self.distance_fromAms = distFromAms 
        self.coords = coords
        self.assigned_hubs = {}
        
    def step(self): 
        self.find_assigned_hubs() 
        if self.assigned_hubs: 
            self.move_materials() 
    
    def find_assigned_hubs(self): 
        self.assigned_hubs = {}
        if self.model.hub_network == 'centralized': 
            hubs = self.model.hubs
        elif self.model.hub_network == 'decentralized': 
            hubs = [hub for hub in self.model.hubs if hub.hubType == 'macro']
        elif self.model.hub_network == 'none': 
            hubs = [site for site in self.model.construction_sites]
        for hub in hubs: 
            hub_id = hub.unique_id
            mat_req = hub.material_request 
            if self.material in mat_req.keys(): 
                self.assigned_hubs[hub_id] = mat_req[self.material]
                    
    def move_materials(self): 
        for hub_id, mat_amount in self.assigned_hubs.items(): 
            vehicle = [v for v in self.model.vehicles_toSupplier if v.hub_id == hub_id][0]
            capacity = vehicle.capacity[self.material]
            emissions_perKm = vehicle.emissions_perKm
            nTrips = mat_amount / capacity 
            emissions = self.distance_fromAms * emissions_perKm * nTrips 
            vehicle.emissions += emissions 
        
class Truck_toSite(Agent):
    def __init__(self, unique_id, model, hub_id, truck_type, capacity, emissions_perKm):
        super().__init__(unique_id, model)
        self.emissions = 0
        self.hub_id = hub_id
        self.truck_type = truck_type
        self.capacity = capacity
        self.emissions_perKm = emissions_perKm
        
    def step(self):
        pass
    
class Vehicle_toSupplier(Agent): 
    def __init__(self, unique_id, model, hub_id, network_type, capacity, emissions_perKm): 
        super().__init__(unique_id, model)
        self.emissions = 0 
        self.hub_id = hub_id
        self.network_type = network_type
        self.capacity = capacity
        self.emissions_perKm = emissions_perKm
        
    def step(self): 
        pass 

In [475]:
from mesa import Model
class Model(Model):
    def __init__(self, parameters_dict): 
        '''create construction sites, hubs, and vehicles'''
        super().__init__()
        self.load_data()
        self.add_parameters(parameters_dict) 
        
        self.id_count = 0
        self.create_constructionSites()
        if self.hub_network != 'none': 
            self.create_hubs()
        self.create_vehicles()
        self.create_suppliers() 
        
        if self.hub_network != 'none': 
            self.create_od_matrix_h2c()
            self.create_od_matrix_h2h()
            self.assign_hubs_to_sites()
            self.assign_hubs_to_hubs()
                    
    def load_data(self): 
        self.schedule = BaseScheduler(self)
        self.construction_sites_df = gpd.read_file('data/data_cleaned/construction_sites.shp')
        self.hubs_df = gpd.read_file('data/data_cleaned/hubs.shp')
        self.suppliers_df = gpd.read_file('data/data_cleaned/suppliers.shp')
        self.vehicles_info = pd.read_csv('data/data_cleaned/vehicles_info.csv')
        self.build_info = pd.read_csv('data/data_cleaned/buildingType_info.csv')
        self.material_list = ['timber', 'concrete', 'modules']
        
        self.construction_sites = []
        self.hubs = []
        self.trucks_toSite = []
        self.vehicles_toSupplier = []
        self.suppliers = []
        
    def add_parameters(self, parameters_dict): 
        self.network_type = parameters_dict['network_type']
        self.truck_type = parameters_dict['truck_type']
        self.biobased_type = parameters_dict['biobased_type']
        self.modularity_type = parameters_dict['modularity_type']
        self.hub_network = parameters_dict['hub_network']
    
    def create_constructionSites(self): 
        for i, row in self.construction_sites_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            site = ConstructionSite(self.id_count, self, row.buildType, 
                                    coords, row.inA10)
            self.schedule.add(site)
            self.construction_sites.append(site)
            self.id_count += 1 
    
    def create_hubs(self): 
        if self.hub_network == 'centralized': 
            hub_type = ['macro']
        elif self.hub_network == 'decentralized': 
            hub_type = ['micro', 'macro']
        elif self.hub_network == 'none': 
            hub_type = []
        for i, row in self.hubs_df.iterrows(): 
            if row.hub_type in hub_type: 
                coords = (row.geometry.y, row.geometry.x)
                hub = Hub(self.id_count, self, row.hub_type, coords, row.inA10)
                self.schedule.add(hub)
                self.hubs.append(hub)
                self.id_count += 1 
            else: 
                None
    
    def create_vehicles(self): 
        if self.hub_network != 'none': 
            hub_ids = [hub.unique_id for hub in self.hubs]
        else: 
            hub_ids = [site.unique_id for site in self.construction_sites]
        
        def get_capacity(network_type, truck_type): 
            v = self.vehicles_info.copy()
            v = v[(v.transType == network_type) & (v.vehicleType == truck_type)]
            capacity_dict = {}
            for mat in self.material_list + ['modules']: 
                capacity = v[f'capacity_{mat}'].iloc[0]
                capacity_dict[mat] = capacity
            emissions_perKm = v.emissions_perKm.iloc[0]
            return capacity_dict, emissions_perKm
        
        # add truck that goes between hub and site
        for hub_id in hub_ids: 
            capacity_dict, emissions_perKm = get_capacity('road', self.truck_type)
            truck_toSite = Truck_toSite(
                self.id_count, self, hub_id, self.truck_type, 
                capacity_dict, emissions_perKm)
            self.schedule.add(truck_toSite)
            self.trucks_toSite.append(truck_toSite)
            self.id_count += 1 

        # add vehicle that goes between hub and supplier
        for hub_id in hub_ids: 
            vehicle_type = 'diesel' if self.network_type == 'road' else self.network_type
            capacity_dict, emissions_perKm = get_capacity(self.network_type, vehicle_type)
            vehicle_toSupplier = Vehicle_toSupplier(
                self.id_count, self, hub_id, self.network_type, 
                capacity_dict, emissions_perKm)
            self.schedule.add(vehicle_toSupplier)  
            self.vehicles_toSupplier.append(vehicle_toSupplier)
            self.id_count += 1 
    
    def create_suppliers(self): 
        for i, row in self.suppliers_df.iterrows(): 
            coords = (row.geometry.y, row.geometry.x)
            supplier = Supplier(self.id_count, self, row.material, 
                                row.distAms, coords)
            self.schedule.add(supplier)
            self.suppliers.append(supplier)
            self.id_count += 1
    
    def create_od_matrix_h2c(self): 
        self.od_matrix_h2c = []
        for site in self.construction_sites: 
            for hub in self.hubs: 
                distance = haversine(site.coords, hub.coords)
                self.od_matrix_h2c.append([site.unique_id, hub.unique_id, distance])
        self.od_matrix_h2c = np.array(self.od_matrix_h2c)
    
    def create_od_matrix_h2h(self): 
        self.od_matrix_h2h = []
        macroHubs = [hub for hub in self.hubs if hub.hubType == 'macro']
        for macroHub in macroHubs: 
            for hub in self.hubs: 
                distance = haversine(macroHub.coords, hub.coords)
                self.od_matrix_h2h.append([hub.unique_id, macroHub.unique_id, distance])
        self.od_matrix_h2h = np.array(self.od_matrix_h2h)
        
    def assign_hubs_to_sites(self):
        od = self.od_matrix_h2c
        nMacroHubs = len([hub for hub in self.hubs if hub.hubType == 'macro'])
        for site in self.construction_sites: 
            site_od = od[od[:, 0] == site.unique_id]
            if self.hub_network == 'decentralized': 
                site_od_micro = site_od[nMacroHubs+1:]
                site.nearestMicroHub_id = int(site_od_micro[np.argmin(site_od_micro[:, 2]), 1])
                site.nearestMicroHub_dist = site_od_micro[np.argmin(site_od_micro[:, 2]), 2]
            site_od_macro = site_od[:nMacroHubs]
            site.nearestMacroHub_id = int(site_od_macro[np.argmin(site_od_macro[:, 2]), 1])
            site.nearestMacroHub_dist = site_od_macro[np.argmin(site_od_macro[:, 2]), 2]
            
    def assign_hubs_to_hubs(self): 
        od = self.od_matrix_h2h
        for hub in self.hubs: 
            hub_od = od[od[:, 0] == hub.unique_id]
            hub.nearestMacroHub_id = int(hub_od[np.argmin(hub_od[:, 2])][1])
            hub.nearestMacroHub_dist = hub_od[np.argmin(hub_od[:, 2])][2]
            
    def step(self):
        self.schedule.step()
    
    def visualize(self): 
        self.display_total_emissions()
        
        m = folium.Map([52.377231, 4.899288], zoom_start=12, tiles='cartodbdark_matter')
        if self.hub_network != 'none': 
            self.plotLines_h2c(m) 
        self.plotLines_s2h(m)
        self.plotPoints(m, [hub for hub in self.hubs if hub.hubType == 'micro'], 'red', 1)
        self.plotPoints(m, [hub for hub in self.hubs if hub.hubType == 'macro'], 'red', 5)
        self.plotPoints(m, self.construction_sites, 'white', 1)
        self.plotPoints(m, self.suppliers, 'pink', 1)
        return m 
    
    def display_total_emissions(self): 
        emissions_h2c = sum([truck.emissions for truck in self.trucks_toSite])
        emissions_s2h = sum([vehicle.emissions for vehicle in self.vehicles_toSupplier])
        total_emissions = round(emissions_h2c + emissions_s2h)
        print(f'emissions (hubs to construction sites): {round(emissions_h2c)} tCO2eq')
        print(f'emissions (suppliers to hubs): {round(emissions_s2h)} tCO2eq')
        print(f'emissions (total): {total_emissions} tCO2eq')
    
    def plotPoints(self, m, agent_list, color, radius): 
        for coord in [agent.coords for agent in agent_list]: 
            folium.CircleMarker(
                location=coord, radius=radius, color=color
            ).add_to(m)

    def plotLines_h2c(self, m): 
        for hub in self.hubs: 
            coords_hub = hub.coords
            coords_sites = [list(site.coords) for site in hub.assigned_sites]
            for coords_site in coords_sites: 
                folium.PolyLine(
                    locations=[coords_site, coords_hub], weight=1, 
                    color='grey', dash_array='5'
                ).add_to(m)
                
    def plotLines_s2h(self, m): 
        if self.hub_network != 'none': 
            hubs = [hub for hub in self.hubs if hub.hubType == 'macro']
        else: 
            hubs = self.construction_sites
        for supplier in self.suppliers: 
            for hub in hubs: 
                folium.PolyLine(
                    locations=[supplier.coords, hub.coords], weight=1, 
                    color='#454545', dash_array='5'
                ).add_to(m)

In [482]:
# input parameters below
parameters_dict = {
    'hub_network': 'decentralized', 
    'network_type': 'rail', 
    'truck_type': 'diesel', 
    'biobased_type': 'semi', 
    'modularity_type': 'full'
}

# create and run model 
model = Model(parameters_dict)
for i in range(15): 
    model.step()

# visualize results 
model.visualize()

emissions (hubs to construction sites): 45 tCO2eq
emissions (suppliers to hubs): 714 tCO2eq
emissions (total): 759 tCO2eq
